# Combining Solar Panels

This study explores the effect of variability in exposure and panel characteristics of PV panels in an array on the array's yield.



## Uniform Case (Control)
### Series
### Parallel
### Dedicated MPPT

## Variation In Exposure Angle
### Series
### Parallel
### Dedicated MPPT

## Variation in Shading
### Series
### Parallel
### Dedicated MPPT

## Variation in Panel Make/Model
### Series
### Parallel
### Dedicated MPPT